# Gated Recurrent Units explained using Matrices
by: Sparkle Russell-Puleri and Dorian Puleri

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import numpy as np
import itertools
import pickle

%autosave 180

Autosaving every 180 seconds


Often times we get consumed with using Deep learning frameworks that perform all of the required operations needed to build our models. However, there is some value to first understanding some of the basic matrix operations used under the hood. In this tutorial we will walk you through the simple matrix operations needed to understand how a GRU works.

### What is a Gated Recurrent Unit (GRU)?
Gated Recurrent Unit (pictured below), is a type of Recurrent Neural Network that addresses the issue of long term dependencies which can lead to vanishing gradients in large networks. GRUs address this issue by storing "memory" from the previous time point to help inform the network for future predictions. At first glance, one may think that this diagram is quite complex, but it is quite the  contrary. The intent of this tutorial is to debunk the difficulty of GRUs using Linear Algebra fundamentals.

<img src="img/gru.png" style="height:400px">

The governing equations for GRUs are: 

<p style='text-align: justify;'> 
<h1><center>$z = \sigma(W_z \boldsymbol{\cdot} x_t + U_z\boldsymbol{\cdot} h_{(t-1)} + b_z)$<br>
$r = \sigma(W_r \boldsymbol{\cdot} x_t + U_r\boldsymbol{\cdot} h_{(t-1)} + b_r)$<br>
$\tilde{h} = tanh(W_h \boldsymbol{\cdot} x_t + r * U_h\boldsymbol{\cdot} h_{(t-1)} + b_z)$<br>
$h = z * h_{(t-1)} + (1 -z) * \tilde{h}$ </p>

where z and r represent the update and reset gates respectively. While $\tilde{h}$ and h represent the intermedidate memory and output respectively.

### GRUs vs Longterm Short Term Memory (LSTM) RNNs
The main differences between GRUs and the popular LSTMs (nicely explained by Chris Olah) are the number of gates and maintenance of cell states. Unlike GRUs, LSTMs have 3 gates (input, forget, output) and maintains an internal memory cell state, which makes it more flexible, but less efficient memory and time wise. However, since both of these networks are great at addressing the vanishing gradient problem required for efficiently tracking long term dependencies. Choosing between them are usually done using a rule of thumb. As such, it is recommended that you first train a LSTM, since it has more parameters and is a bit more flexible, followed by a GRU, and if there are no sizable differences between the performance of the two, then use the much simpler and efficient GRU.

### Approach
To further illustrate the elegance of RNNs, we are going to walk you through the basics of linear algebra needed to understand the inner workings of a GRU. To do this we will use a small string of letters to illustrate exactly how the matrix calculations we take for granted, using pre-packaged wrapper functions, created many of the common DL frameworks. The point of this tutorial is not to set us back, but to help drive a deeper understanding of how RNNs work using Linear Algebra.

Sample using the following sample string as our input data:

`text = MathMathMathMathMath`

However, alogrithms are essentially mathematical equations of some sort, therefore our original text will have to be represented in numerical form before presenting it to the GRU layers. This is done in the following pre-processing step below.

### Data Pre-processing

In [2]:
# This will be our input ---> x
text = 'CellCellCellCellCell'

In the first step a dictionary of all of the unique characters is created to map each letter to a unique integer:

Character dictionary : {'h': 0, 'a': 1, 't': 2, 'M': 3\} * dictionaries are inherently unordered so this will change every time you run the notebook

Our encoded input now becomes:
MathMath = [3, 1, 2, 0, 3, 1, 2, 0]

In [3]:
character_list = list(set(text))   # get all of the unique letters in our text variable
vocabulary_size = len(character_list)   # count the number of unique elements
character_dictionary = {'C': 0, 'e': 1, 'l': 2}
# {char:e for e, char in enumerate(character_list)}  # create a dictionary mapping each unique char to a number
encoded_chars = [character_dictionary[char] for char in text] #integer representation of our vocabulary 

In [4]:
print(character_dictionary)

{'C': 0, 'e': 1, 'l': 2}


In [5]:
print(encoded_chars)

[0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2]


### Step 1: Create batches of data
This step is achieved by user specifying how many batches (B), or sequence length (S) given our vocabulary (V), do we want to create. The figure below demonstrates how bacthes are created and encoded.



Let's assume we want the following parameters:
<ol>
  <li> Batch size (B) = 2</li>
  <li> Sequence length (S) = 3</li>
  <li> Vocabulary (V) = 4</li>
  <li> Output (O) = 4</li>
  <li> Number of samples (NS) = 2</li>
    
</ol>

<img src="img/data_prep.png" style="height:600px">

### So what is the time series?
If you do a basic search for an RNN the image below is typically what you will find. This image is a generalized view of what happens in unfolded form. However, what does the $x_{(t-1)}$, $x_{(t)}$  and $x_{(t+1)}$ (highlighed in red)  mean in terms of our batches?

<img src="img/vanilla_rnn.png" style="height:220px">

In the case of our mini-batch, the time series represents each sequence with information flowing from left to right as shown in the figure below.

<img src="img/gru_fin.png" style="height:400px">

### Dimensions of our dataset

<img src="img/dimensions.png" style="height:470px">

### Step 1: Illustrated in code

In [6]:
def one_hot_encode(encoded, vocab_size):
    result = torch.zeros((len(encoded), vocab_size))
    for i, idx in enumerate(encoded):
        result[i, idx] = 1.0
    return result

### One-hot encoding training data

In [7]:
# One hot encode our encoded charactes
batch_size = 2
seq_length = 3
num_samples = (len(encoded_chars) - 1) // seq_length # time lag of 1 for creating the labels
vocab_size = 3

data = one_hot_encode(encoded_chars[:seq_length*num_samples], vocab_size).reshape((num_samples, seq_length, vocab_size))
num_batches = len(data) // batch_size
X = data[:num_batches*batch_size].reshape((num_batches, batch_size, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
X = X.transpose(1, 2)

After reshaping, if you check the shape of X you would find that you get a rank 3 tensor of shape: 3 x 3 x 2 x 4. What does this mean?

<img src="img/dim_batches.png" style="height:170px">

In [8]:
X.shape 

torch.Size([3, 3, 2, 3])

In [9]:
# However, we typically need the batch to be first so we would need to reshape our dataset --. S x B x V
X, X.shape

(tensor([[[[1., 0., 0.],
           [0., 0., 1.]],
 
          [[0., 1., 0.],
           [1., 0., 0.]],
 
          [[0., 0., 1.],
           [0., 1., 0.]]],
 
 
         [[[0., 0., 1.],
           [0., 1., 0.]],
 
          [[0., 0., 1.],
           [0., 0., 1.]],
 
          [[1., 0., 0.],
           [0., 0., 1.]]],
 
 
         [[[1., 0., 0.],
           [0., 0., 1.]],
 
          [[0., 1., 0.],
           [1., 0., 0.]],
 
          [[0., 0., 1.],
           [0., 1., 0.]]]]),
 torch.Size([3, 3, 2, 3]))

### One-hot encoding labels

In [10]:
# +1 shift the labels by one so that given the previous letter the char we should predict would be or next char
labels = one_hot_encode(encoded_chars[1:seq_length*num_samples+1], vocab_size) 
y = labels.reshape((num_batches, batch_size, seq_length, vocab_size))
y = y.transpose(1, 2) # transpose the first and second index
y,y.shape

(tensor([[[[0., 1., 0.],
           [1., 0., 0.]],
 
          [[0., 0., 1.],
           [0., 1., 0.]],
 
          [[0., 0., 1.],
           [0., 0., 1.]]],
 
 
         [[[0., 0., 1.],
           [0., 0., 1.]],
 
          [[1., 0., 0.],
           [0., 0., 1.]],
 
          [[0., 1., 0.],
           [1., 0., 0.]]],
 
 
         [[[0., 1., 0.],
           [1., 0., 0.]],
 
          [[0., 0., 1.],
           [0., 1., 0.]],
 
          [[0., 0., 1.],
           [0., 0., 1.]]]]),
 torch.Size([3, 3, 2, 3]))

### What will be demonstrated 
The data is now ready for modeling. However, we want to highlight the flow of this tutorial. We will demonstrate the matrix operations performed for the first sequence(highlighed in red) within batch 1 (shown below). The idea is to understand how the information from the first sequence gets passed to the second sequence and so on.

<img src="img/batch1_seq1.png" style="height:70px">

To do this we need to first recall how these batches are fed into the algorithm.

<img src="img/batch_1_gru.png" style="height:350px">

More specifically we will walkthrough all of the matrix operations done in a GRU cell for sequence 1 and the resulting outputs $y_{(t-1)}$ and $h_t$ will be calculated in the process (shown below):

<img src="img/seq1_gru.png" style="height:380px">

### Step 2: Define our weights matrices and bias vectors
In this step we will walk you through the matrix operations used to calculate the z gate, since the calculations are exactly the same for the remaining three equations. But first, recall the governing equations of a GRU:

<p style='text-align: justify;'> 
<h1><center>$z = \sigma(W_z \boldsymbol{\cdot} x_t + U_z\boldsymbol{\cdot} h_{(t-1)} + b_z)$<br>
$r = \sigma(W_r \boldsymbol{\cdot} x_t + U_r\boldsymbol{\cdot} h_{(t-1)} + b_r)$<br>
$\tilde{h} = tanh(W_h \boldsymbol{\cdot} x_t + r * U_h\boldsymbol{\cdot} h_{(t-1)} + b_z)$<br>
$h = z * h_{(t-1)} + (1 -z) * \tilde{h}$ </p>

To help drive this point home we are going to walk through the dot product of the reset gate z by breaking the inner equation down into three sections and finally we will apply the sigmoid activation function to the output to squish the values between 0 and 1:

<img src="img/z_exp.png" style="height:50px">

But first let's define the network parameters:

In [11]:
torch.manual_seed(1) # reproducibility

####  Define the network parameters:
hiddenSize = 2 # network size, this can be any number (depending on your task)
numClass = 3 # this is the same as our vocab_size

#### Weight matrices for our inputs 
Wz = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=True)
Wr = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=True)
Wh = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=True)

## Intialize the hidden state
# this is for demonstration purposes only, in the actual model it will be initiated during training a loop over the 
# the number of bacthes and updated before passing to the next GRU cell.
h_t_demo = torch.zeros(batch_size, hiddenSize) 

#### Weight matrices for our hidden layer
Uz = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=True)
Ur = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=True)
Uh = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=True)

#### bias vectors for our hidden layer
bz = Variable(torch.zeros(hiddenSize), requires_grad=True)
br = Variable(torch.zeros(hiddenSize), requires_grad=True)
bh = Variable(torch.zeros(hiddenSize), requires_grad=True)

#### Output weights
Wy = Variable(torch.randn(hiddenSize, numClass), requires_grad=True)
by = Variable(torch.zeros(numClass), requires_grad=True)

Let's breakdown the dimensions of the networks parameters.<br>
**Input weights dimensions:**<br>
$x_t = (S\times B\times V)\Longrightarrow 3\times 2\times 4 $<br>
$W_{z} = V \times H \Longrightarrow 4 \times 2$<br>
$W_{r} = V \times H\Longrightarrow 4 \times 2$<br>
$W_{h} = V \times H\Longrightarrow  4 \times 2$ <br>

**Hidden weights dimensions:**<br>
$U_{z} = I \times H \Longrightarrow 4 \times 2$<br>
$U_{r} = I \times H\Longrightarrow 4 \times 2$<br>
$U_{h} = I \times H\Longrightarrow  4 \times 2$<br>

**Bias vectors dimensions:**<br>
$b_{z} = H \Longrightarrow  2$<br>
$b_{r} = H \Longrightarrow 2$<br>
$b_{h} = H \Longrightarrow 2$<br>

**Output dimesions***<br>
$h \Longrightarrow 2\times 2 $<br>
$y_{(t+...)} \Longrightarrow H \times O$<br>
*Demonstrated in the broadcasting section

### What is a hidden size?
The hidden size defined above, is the number of learned parameters or simply put, the networks memory. This parameter is usually defined by the user depending on the problem at hand as using more units can make it more likely to over fit the training data. In our case we chose a hidden size of 2 to make this easier to illustrate. These values are often initialized to random numbers from the normal distribution, which are trainable and updated as we perform our forward passes and backpropagation.

<img src="img/wz.png" style="height:200px">

In [12]:
print(f'Weight Matrix for Wz:\n {Wz,Wz.size()}','\n')
print(f'Weight Matrix for Uz:\n {Uz, Uz.size()}','\n')
print(f'Weight Matrix for h_t_demo:\n {h_t_demo,h_t_demo.size()}','\n')
print(f'Bias vector for bz: \n {bz, bz.size()}','\n')

Weight Matrix for Wz:
 (tensor([[ 0.6614,  0.2669],
        [ 0.0617,  0.6213],
        [-0.4519, -0.1661]], requires_grad=True), torch.Size([3, 2])) 

Weight Matrix for Uz:
 (tensor([[-1.5727, -0.1232],
        [ 3.5870, -1.8313]], requires_grad=True), torch.Size([2, 2])) 

Weight Matrix for h_t_demo:
 (tensor([[0., 0.],
        [0., 0.]]), torch.Size([2, 2])) 

Bias vector for bz: 
 (tensor([0., 0.], requires_grad=True), torch.Size([2])) 



### Dimensions of our weights

We will walkthrough all of the matrix operations using the first bacth, as it's exactly the same process for all other batches. However, before we begin any of the above matrix operations, lets discuss an important concept called broadcasting. If we look at the shapes of batch 1 (3 x 2 x 4) and the shape of Wz (4 x 2), the first thing that may come to mind is, how would we perform element-wise matrix multiplication on these two tensors with different shapes?

The answer is we use a process called "Broadcasting". Broadcasting is used to make the shapes of the these two tensors compatitible, such that we can perform our element-wise matrix operations. This means that $W_z$ will get broadcasted to a the non-matrix dimensions, which in our case is our sequence length of 3. This then means that all of the other terms in the update equations z will also get broacasted. Therefore, our final equation will look like this:

<img src="img/broadcast_z.png" style="height:80px">

Before we perform the actual matrix arithmetic let's visualize what sequence 1 from batch one looks like:

<img src="img/z_seq1.png" style="height:430px">

### The update gate: z
The update gate determines how useful past information is to the current state. Here, the use of the sigmoid function results in update gate values between 0 and 1. Therefore ,the closer this value is to 1 the more we incorporate past information , while values closer to 0 would mean that only new information is kept.

Now let’s get to the math…
First term: Note that when these two matrices are multiplied using the dot product, we are multiplying each row by each column. Here, each row (highlighted in yellow) of the first matrix ( x_t) gets multiplied element-wise by each column (highlighted in blue) of the second matrix (Wz).

Term 1: Weights applied to the inputs

In [13]:
torch.matmul(X[0][0], Wz) # This can be easily handled in pytorch using the mm or matmul(handles broadcasting)

tensor([[ 0.6614,  0.2669],
        [-0.4519, -0.1661]], grad_fn=<MmBackward>)

<img src="img/w_dot_x.png" style="height:250px">

Term 2: Hidden Weights

In [14]:
torch.matmul(Uz, h_t_demo), Uz

(tensor([[0., 0.],
         [0., 0.]], grad_fn=<MmBackward>),
 tensor([[-1.5727, -0.1232],
         [ 3.5870, -1.8313]], requires_grad=True))

<img src="img/u_ht_1.png" style="height:180px">

Term 3: Bias Vector

In [15]:
bz

tensor([0., 0.], requires_grad=True)

<img src="img/bz.png" style="height:150px">

### Putting it all together : $z_{inner}$

In [16]:
z_inner = torch.matmul(X,Wz) +  torch.matmul(Uz,h_t_demo) + bz
z_inner[0][0]

tensor([[ 0.6614,  0.2669],
        [-0.4519, -0.1661]], grad_fn=<SelectBackward>)

<img src="img/z_inner.png" style="height:100px">

The values in the resulting matrix is then squished between 0 and 1 using the sigmoid activation function:<br><br>
<h1><center>$\sigma = \frac{1}{1 + exp^{-input}}$

In [17]:
z = torch.sigmoid(z_inner)
z[0][0]

tensor([[0.6596, 0.5663],
        [0.3889, 0.4586]], grad_fn=<SelectBackward>)

<img src="img/sig_z.png" style="height:67px">

### The reset gate: r
Reset gate allows the model to ignore past information that might be irrelevant in future time-steps. Over each batch, the reset gate will re-evaluate the combined performance of prior and new inputs and reset as needed for the new inputs. Again because of the sigmoid activation function, values closer to 0 would mean that we would keep ignore the previous hidden state, and the opposite is true for values closer to 1.

In [18]:
r = torch.sigmoid(torch.matmul(X,Wr) +  torch.matmul(Ur,h_t_demo) + br)
r[0][0]  # first bactch sequence 

tensor([[0.1791, 0.5943],
        [0.2906, 0.4854]], grad_fn=<SelectBackward>)

<img src="img/r.png" style="height:100px">

### Intermediate Memory: $\tilde{h}$
The intermediate memory unit or candidate hidden state combines the information from the previous hidden state with the input.Since the matrix operations required for the first and third terms are the same as what we did in z, we will only present the results.

<img src="img/h_inter_eq.png" style="height:50px">

First term: 

In [19]:
torch.matmul(X[0][0], Wh)

tensor([[-0.1955, -0.9656],
        [-0.4212, -0.5107]], grad_fn=<MmBackward>)

Second term:

In [20]:
torch.matmul(r[0][0] * h_t_demo, Uh)

tensor([[0., 0.],
        [0., 0.]], grad_fn=<MmBackward>)

<img src="img/r_Uh_h_t.png" style="height:150px">

Third term: Bias vector

In [21]:
bh

tensor([0., 0.], requires_grad=True)

<img src="img/bh.png" style="height:130px">

### Putting it all together: $\tilde{h}$

<img src="img/h_tilde_inner.png" style="height:90px">

The values in the resulting matrix is then squished between 0 and 1 using the tanh activation function: <br>
<h1><center>$tanh = \frac{exp^{input} - exp^{-input}}{exp^{input} + exp^{-input}}$

In [22]:
h_inner_tilde = torch.matmul(X,Wh) +  r*torch.matmul(Uh,h_t_demo) + bh
h_inner_tilde[0][0]

tensor([[-0.1955, -0.9656],
        [-0.4212, -0.5107]], grad_fn=<SelectBackward>)

In [23]:
h_tilde = torch.tanh(h_inner_tilde)
h_tilde[0]

tensor([[[-0.1931, -0.7468],
         [-0.3979, -0.4705]],

        [[ 0.3990,  0.2611],
         [-0.1931, -0.7468]],

        [[-0.3979, -0.4705],
         [ 0.3990,  0.2611]]], grad_fn=<SelectBackward>)

<img src="img/h_tilde_final.png" style="height:120px">

### Output hidden layer at time step t: $h_{(t-1)}$

<img src="img/h_out.png" style="height:160px">

First term: 

<img src="img/z_out_first_term.png" style="height:120px">

Second term: 

In [24]:
ht_1 = z *h_t_demo + (1-z)* h_tilde
ht_1[0][0]

tensor([[-0.0657, -0.3239],
        [-0.2432, -0.2547]], grad_fn=<SelectBackward>)

<img src="img/ht_second_term.png" style="height:290px">

### Putting it all together: ${h_t}$

<img src="img/ht_out.png" style="height:90px">

### How does the second sequence in batch 1 (time step $x_t$) information from this hidden state?
Recall, that $h_{(t-n)}$ is first initialized to zeros (used in this tutorial) or random noise to begin the training after which the network would learn and adapt. But after the first iteration, the new hidden state $h_t$ will now be used as our new hidden state and the calculations above are repeated for sequence 2 at time step ($x_t$). The image below demonstrates how this is done.

<img src="img/gru_ht.png" style="height:450px">

This new hidden state $h_{(t-1)}$ will not be used to calculate the output ( $y_{(t+1)}$) and hidden state $h_{(t)}$ of the second time step in the batch and so on. 

<img src="img/hidden_gru_image.png" style="height:450px">

Below we demonstrate how the new hidden state $h_{(t-1)}$ is used to calculate subsequent hidden states. This is typically done using a loop. This loop iterates over all of the elements within each the given batch to calculate both $h_{(t-1)}$.

### Code Implementation: Batch 1 outputs: $h_{(t-1)}$, $h_t$ and $h_{(t+1)}$

In [25]:
# h gets updated and then we calculate for the next 
h_t_1 = []
h = h_t_demo
for i,sequence in enumerate(X[0]):   # iterate over each sequence in the batch to calculate the hidden state h 
    z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
    r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
    h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
    h = z * h + (1 - z) * h_tilde
    h_t_1.append(h)
    print(f'h{i}:{h}')
h_t_1 = torch.stack(h_t_1)

h0:tensor([[-0.0657, -0.3239],
        [-0.2432, -0.2547]], grad_fn=<AddBackward0>)
h1:tensor([[ 0.2255, -0.1708],
        [-0.2692, -0.3959]], grad_fn=<AddBackward0>)
h2:tensor([[-0.2202, -0.1984],
        [ 0.1196, -0.2496]], grad_fn=<AddBackward0>)


### Illustration of using the new hidden state $h_{(t-1)}$  to calculate: $h_t$
Using the second sequence in batch 1 we can demonstrate how we you can visualize how the $h_1$ above for the second sequence was obtained.

In [26]:
h_t_minus_1 = torch.tensor([[ 0.7565, -0.3472],
        [-0.1355, -0.2040]])

In [27]:
X[0][0], X[0][1] # second sequence in batch 1

(tensor([[1., 0., 0.],
         [0., 0., 1.]]),
 tensor([[0., 1., 0.],
         [1., 0., 0.]]))

In [28]:
h = h_t_minus_1
z = torch.sigmoid(torch.matmul(X[0][1], Wz) + torch.matmul(h, Uz) + bz)
r = torch.sigmoid(torch.matmul(X[0][1], Wr) + torch.matmul(h, Ur) + br)
h_tilde = torch.tanh(torch.matmul(X[0][1], Wh) + torch.matmul(r * h, Uh) + bh)
hh = z * h + (1 - z) * h_tilde
hh

tensor([[ 0.7340, -0.0579],
        [-0.1980, -0.3758]], grad_fn=<AddBackward0>)

### What will be the hidden state for the second batch?
If you are a visual person, it can be seen as a series the output at $h_{(t+1)}$, will then be feed to the next bacth and the whole process begins again.

<img src="img/batch1_batch2.png" style="height:350px">

### Code Implementation: Passing hidden states across batches

In [29]:
ht_2 = [] # stores the calculated h for each input x
h = torch.zeros(batch_size, hiddenSize) # intitalizes the hidden state
for batch in range(num_batches):  # this loops over the batches 
    x = X[batch]
    for sequence in x: # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde
        ht_2.append(h)
ht_2 = torch.stack(ht_2)
ht_2

tensor([[[-0.0657, -0.3239],
         [-0.2432, -0.2547]],

        [[ 0.2255, -0.1708],
         [-0.2692, -0.3959]],

        [[-0.2202, -0.1984],
         [ 0.1196, -0.2496]],

        [[-0.4057, -0.3210],
         [ 0.3316, -0.0792]],

        [[-0.4994, -0.3663],
         [-0.1166, -0.1107]],

        [[-0.4299, -0.4447],
         [-0.3371, -0.2914]],

        [[-0.3947, -0.4876],
         [-0.4710, -0.3541]],

        [[ 0.0724, -0.3389],
         [-0.4125, -0.4390]],

        [[-0.3591, -0.3163],
         [ 0.0473, -0.2933]]], grad_fn=<StackBackward>)

### Step 3: Calculate the out predictions for each time step
To obtain our predictions for each time step we first have to transform our output using a linear layer. Recall the dimensions of columns in the hidden states $h_{(t+n)}$ is essentially the dimension of the network size/hidden size. However, we have 4 unique inputs and we are expecting our outputs to also have a size of 4. Therefore, we use what is called a dense layer or fully connect layer to transform our outputs back to the desired dimensions. This fully connected layer is then passed into an activation function (softmax for this tutorial), depending on the desired output.

<img src="img/softmax_layer.png" style="height:350px">

In [30]:
# This is the same 
fully_connected = torch.matmul(ht_2, Wy) + by
fully_connected[0]

tensor([[ 0.7913, -0.2321,  0.0375],
        [ 0.7963, -0.2864,  0.0084]], grad_fn=<SelectBackward>)

<img src="img/linear_layer.png" style="height:290px">

Finally, we apply the softmax activation function to normalize our outputs into a probability distribution, which sums up to 1. The softmax function:<br> 
$\text{Softmax}(x_{i}) = \frac{exp(x_i)}{\sum_j exp(x_j)}$

In [31]:
fully_connected

tensor([[[ 7.9126e-01, -2.3212e-01,  3.7545e-02],
         [ 7.9628e-01, -2.8642e-01,  8.4218e-03]],

        [[ 1.8103e-01,  1.8673e-02,  4.8485e-02],
         [ 1.1388e+00, -3.8616e-01,  2.5078e-02]],

        [[ 6.4817e-01, -2.3977e-01,  3.1697e-03],
         [ 4.5534e-01, -8.6598e-02,  4.7716e-02]],

        [[ 1.0935e+00, -4.1472e-01, -3.3878e-04],
         [-1.2224e-01,  1.3177e-01,  4.7511e-02]],

        [[ 1.2809e+00, -4.9302e-01, -4.3985e-03],
         [ 3.5590e-01, -1.3037e-01,  2.4533e-03]],

        [[ 1.3951e+00, -5.0298e-01,  1.4109e-02],
         [ 9.6431e-01, -3.5957e-01,  3.1306e-03]],

        [[ 1.4600e+00, -5.0987e-01,  2.3930e-02],
         [ 1.2277e+00, -4.7027e-01, -2.9558e-03]],

        [[ 6.9980e-01, -1.6634e-01,  5.4852e-02],
         [ 1.3663e+00, -4.9004e-01,  1.5244e-02]],

        [[ 1.0407e+00, -3.8665e-01,  4.1594e-03],
         [ 6.1975e-01, -1.5234e-01,  4.5784e-02]]], grad_fn=<AddBackward0>)

In [32]:
fully_connected.max()

tensor(1.4600, grad_fn=<MaxBackward1>)

Depending on the textbook you may see different flavors of the softmax, particulary using the softmax max trick which subtracts the maximum value of the entire dataset to prevent exploding values for large $y_{linear}$/fully_connected. In our case this means that our max value of  0.9021 will first be subtracted from ${y}_{linear} $ prior to applying it the the softmax equation.

Let's break this down, please note that we cannot subset the sequences as we did earlier because the summation requires all elements in the entire batch.

1. Subtract the max value of the entire dataset from all of the elements in the fully connected layer<br>
$exp(y_{linear} - y_{linear_{max}})$ = 

In [33]:
ylin_max = (fully_connected - fully_connected.max()) # first sequence in batch 1
ylin_max[0]

tensor([[-0.6687, -1.6921, -1.4224],
        [-0.6637, -1.7464, -1.4515]], grad_fn=<SelectBackward>)

In [34]:
exp = ylin_max.exp()
exp[0]

tensor([[0.5124, 0.1841, 0.2411],
        [0.5149, 0.1744, 0.2342]], grad_fn=<SelectBackward>)

<img src="img/ylin_max.png" style="height:220px">

2. Find the sum of all of the elements within the matrix of exponents

In [35]:
exp_sum = exp[0].sum(dim=1,keepdim=True).reshape(-1,1) # the max value in each element within the sequence
exp_sum

tensor([[0.9376],
        [0.9236]], grad_fn=<ViewBackward>)

<img src="img/exp_sum.png" style="height:250px">

3. Divide each element in the matrix from step 1 by the values from it's respective row in step 2 

In [36]:
exp[0]/exp_sum

tensor([[0.5465, 0.1964, 0.2572],
        [0.5576, 0.1888, 0.2536]], grad_fn=<DivBackward0>)

<img src="img/softmax.png" style="height:110px">

### Code Implementation: Softmax

In [37]:
ht_2 = [] # stores the calculated h for each input x
outputs = []
h = torch.zeros(batch_size, hiddenSize) # intitalizes the hidden state
for i in range(num_batches):  # this loops over the batches 
    x = X[i]
    for i,sequence in enumerate(x): # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde
        
        # Linear layer
        y_linear = torch.matmul(h, Wy) + by
        
        # Softmax activation function
        y_t = F.softmax(y_linear, dim=1)
        
        ht_2.append(h)
        outputs.append(y_t)
        
ht_2 = torch.stack(ht_2)
outputs = torch.stack(outputs)
outputs

tensor([[[0.5465, 0.1964, 0.2572],
         [0.5576, 0.1888, 0.2536]],

        [[0.3668, 0.3119, 0.3213],
         [0.6468, 0.1408, 0.2124]],

        [[0.5165, 0.2125, 0.2710],
         [0.4451, 0.2589, 0.2961]],

        [[0.6426, 0.1422, 0.2152],
         [0.2878, 0.3711, 0.3411]],

        [[0.6915, 0.1173, 0.1912],
         [0.4316, 0.2654, 0.3031]],

        [[0.7137, 0.1069, 0.1794],
         [0.6066, 0.1614, 0.2320]],

        [[0.7260, 0.1013, 0.1727],
         [0.6779, 0.1241, 0.1980]],

        [[0.5141, 0.2162, 0.2697],
         [0.7066, 0.1104, 0.1830]],

        [[0.6271, 0.1505, 0.2224],
         [0.4937, 0.2281, 0.2781]]], grad_fn=<StackBackward>)

In [38]:
hh = torch.tensor([[ 0.7948, -0.8400],
         [-0.3061, -0.7358]])
fcc = torch.matmul(hh, Wy) + by
F.softmax(fcc, dim=1)

tensor([[0.6011, 0.1715, 0.2274],
        [0.8119, 0.0632, 0.1249]], grad_fn=<SoftmaxBackward>)

### Training our network (forward only)
Here we train the network on the input batches by running each batch through the network several times, which is called an epoch. This allows the network to learn the sequences many times. This is then followed with a loss calculation and backpropagation to minimize our loss. In this section we will implement all of the code snippets showed above in one pass. Given the the small input size we will only demonstrate the forward pass, as the calculation of the loss function and backpropagation will be detailed in a subsequent tutorial. 

In [39]:
def gru(x, h):
    outputs = []
    for i,sequence in enumerate(x): # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde

        # Linear layer
        y_linear = torch.matmul(h, Wy) + by

        # Softmax activation function
        y_t = F.softmax(y_linear, dim=1)

        outputs.append(y_t)
    return torch.stack(outputs), h
    

This function will feed a primer of letters to the network help create an initial states and avoid making random guesses. As shown below the first couple of strings generated are a bit erratic, but after a few passes it seems to get at least the next two characters correct. However, given the small vocabulary size this network is most likely overfitting. 

In [40]:
def sample(primer, length_chars_predict):
    
    word = primer

    primer_dictionary = [character_dictionary[char] for char in word]
    test_input = one_hot_encode(primer_dictionary, vocab_size)
    

    h = torch.zeros(1, hiddenSize)

    for i in range(length_chars_predict):
        outputs, h = gru(test_input, h)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].detach().numpy())
        word += character_list[choice]
        input_sequence = one_hot_encode([choice],vocab_size)
    return word


In [43]:
max_epochs = 10  # passes through the data
for e in range(max_epochs):
    h = torch.zeros(batch_size, hiddenSize)
    for i in range(num_batches):
        x_in = X[i]
        y_in = y[i]
        
        out, h = gru(x, h)
        print(sample('Ce',20))

CeCelCeCeCClCeelClCCel
CeelCCCeCeleleCeCCCeCl
CeCCllCCllClCClCeCelee
CeClelellCCCllClllCeCC
CeCClleeClelelellCClCl
CeCleeelllleeeClCeCelC
CelllCCCleeClCleCCClll
CeelleeCCCeeeleelleeCe
CeeClClCeeClelCeCClleC
CeCeeCeeleleeeCCeelCel
CelCClleeCllCllleCeele
CeCeeCleelelCeCelleell
CelCCeClCllelelleCCCll
CeelCCelleeCleeeleelee
CellClCClllllClCeCelCe
CeleeCCCCeeCCeCllelell
CeeelelCeCCleelelClCCl
CelCeeeelCelelleelllle
CeCCeCeCCeCleClelelllC
CelCllllClleCleeleClCe
CeleClCleeCCeClllellll
CeellCCCllClleCeCeCele
CeeClleCCleeCeCCCeeeCC
CeelClCCCCCllClllellCl
CeClllCeCleeCeelelClll
CeCClllleeeCellelCeCel
CeeCClCeleCCCelCeCCeee
CeeelCellCleeeeClelCee
CeCClCClCelllClClelCCl
CeeleCeClCCCClCeleleee


### Final Words
The intent of this tutorial was to provide a walkthrough of the inner working of GRUs using demonstrating how simple matrix operations when combined can make such a powerful algorithm.

### References:
1. The Unreasonable Effectiveness of Recurrent Neural Networks
2. Udacity Deep Learning with Pytorch
3. Fastai Deep Learning for Coders
4. Deep Learning - The Straight Dope (RNNs)
5. Deep Learning Book